In [695]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

# 1. 데이터 셋 원본 확인

In [696]:
df = pd.read_csv('./csv파일/심각도원본데이터_전국.csv',encoding='cp949')
df = df.drop(df.columns[0], axis='columns')

In [697]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1116035 entries, 0 to 1116034
Data columns (total 58 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   사고번호          1116035 non-null  int64 
 1   발생일시          1116035 non-null  int64 
 2   발생지_시도        1116035 non-null  object
 3   발생지_시군구       1116035 non-null  object
 4   법정동코드         1116035 non-null  int64 
 5   요일            1116035 non-null  object
 6   주야            1116035 non-null  object
 7   사고내용          1116035 non-null  object
 8   사고유형_대분류      1116035 non-null  object
 9   사고유형_중분류      1116035 non-null  object
 10  사고유형          1116035 non-null  object
 11  가해자성별         1116035 non-null  object
 12  피해자성별         1116035 non-null  object
 13  연령가해자         1116035 non-null  object
 14  연령피해자         1116035 non-null  object
 15  가해자음주_대분류     1116035 non-null  object
 16  가해자음주         1116035 non-null  object
 17  보호장구가해자_대분류   1116035 non-null  object
 18  보호

In [698]:
df

,사고번호,발생일시,발생지_시도,발생지_시군구,법정동코드,요일,주야,사고내용,사고유형_대분류,사고유형_중분류,...,기상상태,노면상태_대분류,노면상태,교차로형태_대분류,교차로형태,사망자수,중상자수,경상자수,부상신고자수,사고위치폴리곤
0,2015010100100019,2015010100,강원,춘천시,4211031022,목,야,중상,차대차,측면충돌,...,맑음,포장,건조,교차로,교차로 - 사지,0,1,3,0,"POLYGON((1023551 1986551,1023551 1986600,10236..."
1,2015010100100121,2015010105,강원,춘천시,4211039000,목,야,부상신고,차량단독,공작물충돌,...,맑음,포장,건조,교차로아님,교차로아님,0,0,0,1,"POLYGON((1022551 1981701,1022551 1981750,10226..."
2,2015010200100075,2015010208,강원,춘천시,4211012400,금,주,경상,차대차,추돌,...,맑음,포장,건조,교차로,교차로 - 사지,0,0,2,0,"POLYGON((1021101 1984151,1021101 1984200,10211..."
3,2015010200100205,2015010213,강원,춘천시,4211012100,금,주,경상,차대차,추돌,...,맑음,포장,건조,교차로아님,교차로아님,0,0,1,0,"POLYGON((1019201 1985201,1019201 1985250,10192..."
4,2015010200100417,2015010220,강원,춘천시,4211012100,금,야,경상,차대차,측면충돌,...,맑음,포장,건조,교차로,교차로 - 사지,0,0,1,0,"POLYGON((1019551 1984451,1019551 1984500,10196..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1116030,2019021400100564,2019021422,충북,옥천군,4373031000,목,야,경상,차대차,추돌,...,맑음,포장,건조,교차로아님,교차로아님,0,0,1,0,"POLYGON((1018001 1808901,1018001 1808950,10180..."
1116031,2019022200100348,2019022215,충북,청주시,4311332025,금,주,경상,차대차,기타,...,맑음,포장,건조,교차로아님,교차로아님,0,0,2,0,"POLYGON((988551 1852151,988551 1852200,988600 ..."
1116032,2019030400100044,2019030405,충북,영동군,4374032023,월,야,중상,차대차,추돌,...,맑음,포장,건조,교차로아님,교차로아님,0,2,0,0,"POLYGON((1037751 1802851,1037751 1802900,10378..."
1116033,2019030900100096,2019030905,충북,청주시,4311331035,토,야,중상,차대차,추돌,...,맑음,포장,건조,교차로아님,교차로아님,0,1,0,0,"POLYGON((989851 1846801,989851 1846850,989900 ..."


---

# 2. 데이터 전처리 진행

###  
###  2.1  <보호장구피해자_대분류> => 보행자 & <사고유형_대분류> => 차대사람 / <연령피해자> => 65세 이상으로 전처리
####   ▶    연령 컬럼을 int로 변환하고 연령 중 없음, 불명은 0세로 대체  /  <연령가해자>도 int로 변환하고 없음, 불명 0세로 대체

In [699]:
df["연령피해자"]=df["연령피해자"].str.replace("세","") 
df["연령피해자"]=df["연령피해자"].str.replace("불명","0")   
df["연령피해자"]=df["연령피해자"].str.replace("없음","0") 

In [700]:
df["연령가해자"]=df["연령가해자"].str.replace("세","") 
df["연령가해자"]=df["연령가해자"].str.replace("불명","0")   
df["연령가해자"]=df["연령가해자"].str.replace("없음","0")

In [701]:
df = df.astype({'연령피해자': 'int'})
df = df.astype({'연령가해자': 'int'})

In [702]:
df=df[df['연령피해자']>=65]
df=df[(df['보호장구피해자_대분류']=='보행자') & (df['사고유형_대분류']=='차대사람')]

### 2.2 <발생일시>에서 '시간대'만 추출하기  /  <발생일시> 끝의 2자리가 발생 시각을 의미함
#### ▶ <발생시각> 컬럼 추가하고, <발생일시> 컬럼 삭제

In [703]:
time=[]; # time이라는 빈 리스트에 시간대 저장
for i in range(len(df)):
    date = df.iloc[i,1]
    date = str(date)
    date = date[-2:]
    time.append(date)

In [704]:
df['발생시각']=time # 발생시각 컬럼 추가 
df = df.drop(['발생일시'],axis=1)

### 2.3 사용하지 않거나 의미가 중복되는 컬럼 삭제
### ▶ 사용하지 않음: 사고번호, 주야, 발생지_시도, 발생지_시군구, 법정동코드, 사고위치폴리곤

In [705]:
df = df.drop(['사고번호', '주야', '발생지_시도', '발생지_시군구', '법정동코드', '사고위치폴리곤'],axis=1)

### ▶ 의미가 중복됨
####  [사고유형_대분류 , 사고유형_중분류, 사고유형    ==>   사고유형] 
####  [가해자음주_대분류, 가해자음주    ==>    가해자음주_대분류]
####  [행동유형가해자_대분류, 행동유형가해자_중분류, 행동유형가해자   ==>   행동유형가해자]
####  [당사자종별가해자_대분류, 당사자종별가해자    ==>    당사자종별가해자_대분류]

In [706]:
col = df.columns
del_list = []
for i in range(len(col)):
    c =col[i]
    if c[-2:] == "분류":
        del_list.append(c)
del_list      # 일단 대분류,중분류가 포함된 컬럼은 삭제할 예정이라 대분류,중분류가 포함된 컬럼명 뽑아봄

['사고유형_대분류',
 '사고유형_중분류',
 '가해자음주_대분류',
 '보호장구가해자_대분류',
 '보호장구피해자_대분류',
 '행동유형가해자_대분류',
 '행동유형가해자_중분류',
 '당사자종별가해자_대분류',
 '당사자종별피해자_대분류',
 '차량용도가해자_대분류',
 '차량용도가해자_중분류',
 '차량용도피해자_대분류',
 '차량용도피해자_중분류',
 '도로형태_대분류',
 '도로선형_대분류',
 '도로선형_중분류',
 '노면상태_대분류',
 '교차로형태_대분류']

---

### A) 사고유형

In [707]:
df['사고유형_대분류'].unique()

array(['차대사람'], dtype=object)

In [708]:
df['사고유형_중분류'].unique()

array(['보도통행중', '횡단중', '길가장자리구역통행중', '차도통행중', '기타'], dtype=object)

In [709]:
df['사고유형'].unique()

array(['보도통행중', '횡단중', '길가장자리구역통행중', '차도통행중', '기타'], dtype=object)

In [710]:
df = df.drop(['사고유형_대분류','사고유형_중분류'],axis=1)

---

### B) 가해자음주
#### <가해자음주_대분류> 사용

In [711]:
df['가해자음주_대분류'].unique()

array(['기타불명', '음주운전', '해당 없음', '측정불능', '음주상태'], dtype=object)

In [712]:
df['가해자음주'].unique()

array(['기타불명', '0.15%~0.19%', '0.05%~0.09%', '0.10%~0.14%', '0.20%~0.24%',
       '측정불응', '해당 없음', '측정불능', '0.03%미만', '0.25%~0.29%', '0.030~0.049%',
       '0.35%이상', '0.30%~0.34%'], dtype=object)

In [713]:
df = df.drop(['가해자음주'],axis=1)

#### <가해자음주_대분류> 를 <가해자음주>로 컬럼명 바꾸고 '음주운전'  == '음주상태' 이므로 이름 통합 

In [714]:
df.rename(columns ={'가해자음주_대분류':'가해자음주'},inplace=True)

In [715]:
idx = df[df['가해자음주']=='음주상태'].index
for i in idx:
    df.iloc[:,7][i]=df.iloc[:,7][i].replace('음주상태','음주운전')

---

### ★ <보호 장구> 컬럼들은 보행자 사고 특성 상 필요하지 않을 것으로 예상되어 삭제 ★

In [716]:
df = df.drop(['보호장구가해자_대분류','보호장구가해자','보호장구피해자_대분류','보호장구피해자'],axis=1)

---

### C) 행동유형가해자
#### <행동유형가해자> 사용 / '직진 중' , '직진중'과 같이 의미가 중복되는 것 정리

In [717]:
df['행동유형가해자_대분류'].unique()

array(['운전자', '불명'], dtype=object)

In [718]:
df['행동유형가해자_중분류'].unique()

array(['자동차 등', '불명', '직진관련', '기타', '후진 중', '회전관련', '주행 중 대기', '주정차중'],
      dtype=object)

In [719]:
df['행동유형가해자'].unique()

array(['직진중', '후진중', '좌우회전중', 'U턴중', '불명', '출발중', '주ㆍ정차중 ', '진로변경중', '기타',
       '직진 중', '후진 중', 'U턴 중', '좌회전 중', '우회전 중', '주행 중 대기', '앞지르기 중',
       '주정차중', '진로변경 중', '앞지르기중'], dtype=object)

In [720]:
for i in range(len(df)):
    print(df.iloc[i,9],"+",df.iloc[i,10],"+",df.iloc[i,11])

운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + U턴중
운전자 + 자동차 등 + 직진중
불명 + 불명 + 불명
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
불명 + 불명 + 불명
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
불명 + 불명 + 불명
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 출발중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
불명 + 불명 + 불명
운전자 + 자동차 등 + 좌우회전중
불명 + 불명 + 불명
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중

운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 출발중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 출발중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + U턴중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + U턴중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 

운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 기타 + 기타
운전자 + 기타 + 기타
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 앞지르기 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
불명 + 불명 + 불명
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
불명 + 불명 + 불명
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌우회전중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 주정차중 + 주정차중
운전자 + 후진 중 

운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 기타 + 기타
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + U턴 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 진로변경 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중

운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + U턴 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 출발중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 출발중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 출발중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 출발중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 주ㆍ정차중 
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 주ㆍ정차중 
운전자 + 자동차 등 + 직진

...

운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 기타 + 기타
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
불명 + 불명 + 불명
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
불명 + 불명 + 불명
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 +

운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 기타 + 기타
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 우회전 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + U턴 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
불명 + 불명 + 불명
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운

운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 주ㆍ정차중 
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 직진중
불명 + 불명 + 불명
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 후진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 주ㆍ정차중 
불명 + 불명 + 불명
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 주ㆍ정차중 
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 직진중
운전자 + 자동차 등 + 좌우회전중
운전자 + 자

운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌우회전중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌우회전중
불명 + 불명 + 불명
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌우회전중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌우회전중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌우회전중
운전자 + 기타 + 기타
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전

운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 주행 중 대기 + 주행 중 대기
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 진로변경 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + U턴 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 우회전 중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 회전관련 + 우회전 중
운전자 + 회전관련 + 좌회전 중
운전자 + 주정차중 + 주정차중
운전자 + 직진관련 + 직진 중
운전자 + 직진관련 + 직진 중
운전자 + 후진 중 + 후진 중
운전자 + 직진

In [721]:
df = df.drop(['행동유형가해자_대분류','행동유형가해자_중분류'],axis=1)

In [722]:
idx = df[df['행동유형가해자']=='직진 중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('직진 중','직진중')
    
idx = df[df['행동유형가해자']=='후진 중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('후진 중','후진중')
    
idx = df[df['행동유형가해자']=='U턴 중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('U턴 중','U턴중')
    
idx = df[df['행동유형가해자']=='좌회전 중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('좌회전 중','좌우회전중')
    
idx = df[df['행동유형가해자']=='우회전 중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('우회전 중','좌우회전중')
    
idx = df[df['행동유형가해자']=='주정차중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('주정차중','주ㆍ정차중')
    
idx = df[df['행동유형가해자']=='주ㆍ정차중 '].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('주ㆍ정차중 ','주ㆍ정차중')
    
idx = df[df['행동유형가해자']=='진로변경 중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('진로변경 중','진로변경중')

idx = df[df['행동유형가해자']=='앞지르기 중'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('앞지르기 중','앞지르기중')

#### '기타'와 '불명'은 묶어서 '기타/불명'으로 처리하겠음

In [723]:
idx = df[df['행동유형가해자']=='기타'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('기타','기타/불명')

idx = df[df['행동유형가해자']=='불명'].index
for i in idx:
    df.iloc[:,9][i]=df.iloc[:,9][i].replace('불명','기타/불명')


In [724]:
df['행동유형가해자'].unique()

array(['직진중', '후진중', '좌우회전중', 'U턴중', '기타/불명', '출발중', '주ㆍ정차중', '진로변경중',
       '주행 중 대기', '앞지르기중'], dtype=object)

---

### D) 당사자종별가해자
#### <당사자종별가해자_대분류> 사용

In [725]:
df['당사자종별가해자_대분류'].unique()

array(['자전거', '승용차', '화물차', '불명', '승합차', '이륜차', '원동기장치자전거', '건설기계', '농기계',
       '특수차', '사륜오토바이(ATV)', '개인형이동수단(PM)', '기타'], dtype=object)

In [726]:
df['당사자종별가해자'].unique()

array(['자전거', '소형', '경형', '중형', '불명', '대형', '원동기장치자전거', '건설기계', '사발이',
       '농기계', '승용차', '화물차', '승합차', '이륜차', '특수차', '사륜오토바이(ATV)',
       '개인형이동수단(PM)', '기타'], dtype=object)

In [727]:
df = df.drop(['당사자종별가해자'],axis=1)

#### <당사자종별가해자_대분류> 를 <당사자종별가해자>로 컬럼명 바꿈

In [728]:
df.rename(columns ={'당사자종별가해자_대분류':'당사자종별가해자'},inplace=True)

---

### E) 당사자종별피해자

#### 처음에 <당사자종별피해자>를 '보행자'로 기준으로 삼아 필터링했기 때문에 컬럼 자체를 제거

In [729]:
print(df['당사자종별피해자_대분류'].unique(),df['당사자종별피해자'].unique())

['보행자'] ['보행자']


In [730]:
df = df.drop(['당사자종별피해자_대분류','당사자종별피해자'],axis=1)

---

### F) 차량용도가해자
#### <당사자종별가해자> + <차량용도가해자_대분류>
####                              또는
#### <차량용도가해자_중분류> 를 사용 할 예정이므로
#### <차량용도가해자> 컬럼 삭제

In [731]:
df['차량용도가해자_대분류'].unique()

array(['자전거', '비사업용', '사업용', '기타불명', '이륜차', '원동기장치자전거', '사륜오토바이(ATV)',
       '불명', '기타', '농기계', '개인형이동수단(PM)'], dtype=object)

In [732]:
df['차량용도가해자_중분류'].unique()

array(['자전거', '승용차', '화물차', '택시', '기타불명', '노선버스', '버스', '이륜차', '비사업용기타',
       '원동기장치자전거', '콘크리트믹서', '전세버스', '렌터카', '사업용기타', '사륜오토바이(ATV)',
       '어린이통학버스', '덤프트럭', '기타/건설기계', '불명', '기타', '농기계', '개인형이동수단(PM)',
       '위험물운송', '특수여객(장의)'], dtype=object)

In [733]:
df['차량용도가해자'].unique()

array(['자전거', '승용차', '화물차', '개인택시', '기타/불명', '법인택시', '일반화물', '시내버스', '버스',
       '이륜차', '비사업용기타', '개별화물', '원동기장치자전거', '콘크리트믹서', '전세버스', '렌터카', '기타',
       '사륜오토바이(ATV)', '어린이통학버스', '덤프트럭', '고속버스', '기타/건설기계', '시외버스', '불명',
       '농기계', '개인형이동수단(PM)', '용달화물', '위험물운송', '특수여객(장의)', '마을버스'],
      dtype=object)

In [734]:
df = df.drop(['차량용도가해자'],axis=1)

---

### G) 차량용도피해자
#### (E)와 같은이유로 컬럼 삭제

In [735]:
print(df['차량용도피해자_대분류'].unique(),df['차량용도피해자_중분류'].unique(),df['차량용도피해자'].unique())

['보행자'] ['보행자'] ['보행자']


In [736]:
df = df.drop(['차량용도피해자_대분류','차량용도피해자_중분류','차량용도피해자'],axis=1)

In [737]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58059 entries, 19 to 1115843
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   요일           58059 non-null  object
 1   사고내용         58059 non-null  object
 2   사고유형         58059 non-null  object
 3   가해자성별        58059 non-null  object
 4   피해자성별        58059 non-null  object
 5   연령가해자        58059 non-null  int32 
 6   연령피해자        58059 non-null  int32 
 7   가해자음주        58059 non-null  object
 8   법규위반가해자      58059 non-null  object
 9   행동유형가해자      58059 non-null  object
 10  신체상해정도가해자    58059 non-null  object
 11  신체상해정도피해자    58059 non-null  object
 12  가해자신체상해주부위   58059 non-null  object
 13  피해자신체상해주부위   58059 non-null  object
 14  당사자종별가해자     58059 non-null  object
 15  차량용도가해자_대분류  58059 non-null  object
 16  차량용도가해자_중분류  58059 non-null  object
 17  도로종류         58059 non-null  object
 18  도로형태_대분류     58059 non-null  object
 19  도로형태         58059 non

---

### H) 도로형태
#### <도로형태_대분류> 사용 

In [738]:
df['도로형태_대분류'].unique()

array(['단일로', '교차로', '기타/불명', '기타', '주차장', '불명'], dtype=object)

In [739]:
df['도로형태'].unique()

array(['기타단일로', '교차로내', '횡단보도부근', '교차로부근', '횡단보도상', '기타/불명', '교량위',
       '교차로횡단보도내', '기타', '주차장', '불명', '터널안', '지하차도(도로)내', '고가도로위'],
      dtype=object)

In [740]:
df = df.drop(['도로형태'],axis=1)
df.rename(columns ={'도로형태_대분류':'도로형태'},inplace=True)

---

### I) 도로선형
#### <도로선형>응 사용    /    이때 '평지'와 '평지 '는 동일하므로 '평지' (공백없음)으로 이름 통일

In [741]:
df['도로선형_대분류'].unique()

array(['직선', '커브ㆍ곡각 ', '기타구역'], dtype=object)

In [742]:
df['도로선형_중분류'].unique()

array(['직선', '좌', '기타구역', '우'], dtype=object)

In [743]:
df['도로선형'].unique()

array(['평지', '오르막', '내리막', '기타/서비스구역', '평지  '], dtype=object)

In [744]:
df = df.drop(['도로선형_대분류','도로선형_중분류'],axis=1)

idx = df[df['도로선형']=='평지  '].index
for i in idx:
    df.iloc[:,24][i]=df.iloc[:,24][i].replace('평지  ','평지')

In [745]:
df['도로선형'].unique()

array(['평지', '오르막', '내리막', '기타/서비스구역', '평지  '], dtype=object)

---

### J) 노면상태
#### <노면상태> 컬럼 사용    /    <노면상태_대분류> 삭제

In [746]:
df['노면상태_대분류'].unique()

array(['포장', '비포장'], dtype=object)

In [747]:
df['노면상태'].unique()

array(['서리/결빙', '건조', '젖음/습기', '기타', '적설'], dtype=object)

In [748]:
df = df.drop(['노면상태_대분류'],axis=1)

---

### K) 교차로형태
#### <교차로형태_대분류> 사용    /    <교차로형태> 삭제    =>    <교차로형태_대분류> 를 <교차로형태>로 이름 변경

In [749]:
df['교차로형태_대분류'].unique()

array(['교차로아님', '교차로', '기타불명'], dtype=object)

In [750]:
df['교차로형태'].unique()

array(['교차로아님', '교차로 - 사지', '교차로 - 삼지', '교차로 - 오지이상', '기타불명', '교차로 - 회전'],
      dtype=object)

In [751]:
df = df.drop(['교차로형태'],axis=1)

In [752]:
df.rename(columns ={'교차로형태_대분류':'교차로형태'},inplace=True)

---

---

# 3. 사고심각도 EPDO 컬럼 추가
##  
### EPDO1 = 273/32/2/1
### EPDO2 = 12 /6  /3 /3

In [753]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58059 entries, 19 to 1115843
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   요일           58059 non-null  object
 1   사고내용         58059 non-null  object
 2   사고유형         58059 non-null  object
 3   가해자성별        58059 non-null  object
 4   피해자성별        58059 non-null  object
 5   연령가해자        58059 non-null  int32 
 6   연령피해자        58059 non-null  int32 
 7   가해자음주        58059 non-null  object
 8   법규위반가해자      58059 non-null  object
 9   행동유형가해자      58059 non-null  object
 10  신체상해정도가해자    58059 non-null  object
 11  신체상해정도피해자    58059 non-null  object
 12  가해자신체상해주부위   58059 non-null  object
 13  피해자신체상해주부위   58059 non-null  object
 14  당사자종별가해자     58059 non-null  object
 15  차량용도가해자_대분류  58059 non-null  object
 16  차량용도가해자_중분류  58059 non-null  object
 17  도로종류         58059 non-null  object
 18  도로형태         58059 non-null  object
 19  노인보호구역_여부    58059 non

In [754]:
EPDO =[];
for i in range(len(df)):
    y = (273*df.iloc[i,26])+(32*df.iloc[i,27])+(2*df.iloc[i,28])+(1*df.iloc[i,29])
    EPDO.append(y)
df['EPDO1'] = EPDO    

In [755]:
EPDO =[];
for i in range(len(df)):
    y = (12*df.iloc[i,26])+(6*df.iloc[i,27])+(3*df.iloc[i,28])+(3*df.iloc[i,29])
    EPDO.append(y)
df['EPDO2'] = EPDO    

In [756]:
df

,요일,사고내용,사고유형,가해자성별,피해자성별,연령가해자,연령피해자,가해자음주,법규위반가해자,행동유형가해자,...,기상상태,노면상태,교차로형태,사망자수,중상자수,경상자수,부상신고자수,발생시각,EPDO1,EPDO2
19,화,경상,보도통행중,여,여,18,68,기타불명,안전운전 의무 불이행,직진중,...,맑음,서리/결빙,교차로아님,0,0,1,0,15,2,3
34,토,중상,횡단중,남,여,80,75,기타불명,안전운전 의무 불이행,직진중,...,맑음,건조,교차로아님,0,1,0,0,09,32,6
61,목,경상,길가장자리구역통행중,남,남,18,68,기타불명,안전운전 의무 불이행,직진중,...,맑음,건조,교차로아님,0,0,1,0,19,2,3
76,목,중상,길가장자리구역통행중,남,여,22,89,기타불명,안전운전 의무 불이행,후진중,...,맑음,건조,교차로아님,0,1,0,0,11,32,6
77,목,중상,횡단중,여,여,45,85,기타불명,안전운전 의무 불이행,직진중,...,맑음,건조,교차로아님,0,1,0,0,12,32,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115751,월,경상,길가장자리구역통행중,여,여,53,65,기타불명,안전운전 의무 불이행,직진중,...,맑음,건조,교차로아님,0,0,1,0,16,2,3
1115812,금,경상,보도통행중,여,남,28,71,해당 없음,기타,후진중,...,맑음,건조,교차로,0,0,1,0,17,2,3
1115826,금,중상,차도통행중,여,남,32,76,해당 없음,안전운전 의무 불이행,직진중,...,맑음,건조,교차로아님,0,1,0,0,10,32,6
1115835,토,중상,기타,남,여,75,77,해당 없음,안전운전 의무 불이행,직진중,...,맑음,건조,교차로아님,0,1,0,0,10,32,6


---

# 4. 데이터 전처리 추가 
####   (두 컬럼 중 어느 것이 더 효과적일지 모르겠어서 둘 다 일단 사용해보고자 함)
### <당사자종별가해자_대분류(당사자종별가해자 로 이름 바꿨음)> + <차량용도가해자_대분류> = <가해자차량용도> 컬럼 추가
### <당사자종별가해자_중분류> 사용

In [757]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58059 entries, 19 to 1115843
Data columns (total 33 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   요일           58059 non-null  object
 1   사고내용         58059 non-null  object
 2   사고유형         58059 non-null  object
 3   가해자성별        58059 non-null  object
 4   피해자성별        58059 non-null  object
 5   연령가해자        58059 non-null  int32 
 6   연령피해자        58059 non-null  int32 
 7   가해자음주        58059 non-null  object
 8   법규위반가해자      58059 non-null  object
 9   행동유형가해자      58059 non-null  object
 10  신체상해정도가해자    58059 non-null  object
 11  신체상해정도피해자    58059 non-null  object
 12  가해자신체상해주부위   58059 non-null  object
 13  피해자신체상해주부위   58059 non-null  object
 14  당사자종별가해자     58059 non-null  object
 15  차량용도가해자_대분류  58059 non-null  object
 16  차량용도가해자_중분류  58059 non-null  object
 17  도로종류         58059 non-null  object
 18  도로형태         58059 non-null  object
 19  노인보호구역_여부    58059 non

In [758]:
List=[];
for i in range(len(df)):
    aa = df.iloc[i,14]
    bb = df.iloc[i,15]
    if aa == bb:
        List.append(aa)
    else:
        List.append(bb+"_"+aa)

In [759]:
df['가해자차량용도'] = List

In [760]:
df = df.drop(['당사자종별가해자','차량용도가해자_대분류'],axis=1)

---

---

---

## EPDO2를 기준으로 구간 나누어서 사고 심각도가 높으면 1 / 아니면 0 부여 하는 <심각도> 컬럼 추가

### 3이하는 0 그 이상은 1로 부여

In [761]:
df['EPDO1'].unique()

array([   2,   32,   64,    1,  273,  275,  546,  274,   34,    6,    4,
         35,  305,    3,   66,  369,  277,   33,  130,   96,   36,  578,
         65, 1094,    8,  309,   40,  311,  371,  548,   20,  287,    5,
        307,  337,  341,  132,  821,  433,   38,  279,   97,  819,  276,
         37,  379,  102,   70,   69,   17,   98,   67,  100,   68,  128,
         71,  192,   42,    9,    7,   10,   76,  278,  339,  596],
      dtype=int64)

In [762]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58059 entries, 19 to 1115843
Data columns (total 32 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   요일           58059 non-null  object
 1   사고내용         58059 non-null  object
 2   사고유형         58059 non-null  object
 3   가해자성별        58059 non-null  object
 4   피해자성별        58059 non-null  object
 5   연령가해자        58059 non-null  int32 
 6   연령피해자        58059 non-null  int32 
 7   가해자음주        58059 non-null  object
 8   법규위반가해자      58059 non-null  object
 9   행동유형가해자      58059 non-null  object
 10  신체상해정도가해자    58059 non-null  object
 11  신체상해정도피해자    58059 non-null  object
 12  가해자신체상해주부위   58059 non-null  object
 13  피해자신체상해주부위   58059 non-null  object
 14  차량용도가해자_중분류  58059 non-null  object
 15  도로종류         58059 non-null  object
 16  도로형태         58059 non-null  object
 17  노인보호구역_여부    58059 non-null  object
 18  어린이보호구역_여부   58059 non-null  object
 19  자전거도로_여부     34691 non

In [763]:
심각도 =[];
for i in range(len(df)):
    if 1<=df.iloc[i,30]<=3:
        심각도.append(0)
    else:
        심각도.append(1)
df['심각도'] = 심각도    

In [764]:
print(심각도.count(0), 심각도.count(1))

19182 38877


# ■ 탐색적 데이터 분석을 위한 1차 데이터 셋 완성

In [766]:
df.to_csv('데이터셋_1.csv',encoding='cp949')